# Trabalho 03 – Forecasting com técnica ingênua (Naïve) e rede neural (LSTM)

Este notebook é para o **Trabalho 03** da disciplina de Redes Neurais Artificiais (RNA).

### Enunciado (resumo)

- Usar as **técnicas de forecasting apresentadas em aula** em uma base de dados **diferente** da utilizada em sala.
- Usar uma técnica **ingênua (Naïve)** como **baseline**.
- Explicar a base usada com **poucas palavras**.
- Justificar os **parâmetros** usados para gerar o dataset a partir do array.

A ideia aqui é manter o estilo dos trabalhos anteriores: código simples, com alguns comentários
sobre as decisões que tomei e pequenos testes que fiz no caminho.


## 1. Base de dados usada (explicação rápida)

Em vez de usar um dataset pronto da internet, eu gerei uma **série temporal sintética** que imita,
de forma bem simples, algo como uma medida de consumo ou demanda ao longo do tempo.

A série tem:

- uma **tendência leve** (valores vão subindo aos poucos);
- uma **sazonalidade** aproximada (um ciclo repetindo mais ou menos a cada 24 pontos);
- um pouco de **ruído aleatório**.

Essa escolha ajuda a controlar melhor a série (eu sei o que tem dentro dela) e ainda assim é um
sinal razoável para testar forecasting com baseline Naïve e com rede neural.


In [ ]:
# Bloco de imports principais. Aqui entrou tudo que eu precisei para o trabalho.

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(42)

print("Versão do TensorFlow:", tf.__version__)

## 2. Gerando a série temporal (array base)

Aqui eu gero a série `serie`, que será o meu array base para montar o dataset de forecasting.

Parâmetros principais escolhidos:

- **comprimento da série**: 500 pontos (não é gigante, mas já dá material suficiente para treino/teste);
- **tendência**: um termo linear bem pequeno, só para os valores irem subindo devagar;
- **sazonalidade**: uma senoide com período por volta de 24 pontos;
- **ruído**: um ruído normal com desvio padrão moderado, para não ficar tudo perfeito demais.

Depois eu só ploto a série para ver se ela ficou com uma cara “ok”.


In [ ]:
# comprimento da série
n_pontos = 500

# eixo de tempo simples
t = np.arange(n_pontos)

# tendência leve
tendencia = 0.03 * t

# componente sazonal (período ~24)
sazonalidade = 2.0 * np.sin(2 * np.pi * t / 24)

# ruído aleatório (não quis exagerar muito aqui)
ruido = np.random.normal(loc=0.0, scale=0.5, size=n_pontos)

# série final (algo como uma "demanda" qualquer)
serie = 10 + tendencia + sazonalidade + ruido

plt.figure(figsize=(10, 4))
plt.plot(t, serie, label="série sintética")
plt.xlabel("tempo (índice)")
plt.ylabel("valor")
plt.title("Série temporal sintética (tendência + sazonalidade + ruído)")
plt.grid(True)
plt.legend()
plt.show()

## 3. Gerando o dataset a partir do array (janela deslizante)

Para usar uma rede neural em forecasting, eu preciso transformar a série 1D em um **dataset supervisonado**.
Ou seja, de um array simples como:

\[ x_1, x_2, x_3, \dots, x_T \]

eu crio pares do tipo:

- entrada: \( [x_t, x_{t+1}, \dots, x_{t+J-1}] \)
- saída: \( x_{t+J} \)

onde \( J \) é o **tamanho da janela** (quantos pontos passados eu olho para prever o próximo).

### Justificativa dos parâmetros

- **janela = 24**: escolhi 24 porque na construção da série eu coloquei uma sazonalidade com
  período de aproximadamente 24 pontos. Então olhar os últimos 24 valores faz sentido para
  capturar pelo menos um “ciclo” completo.
- **split treino/teste**: usei os primeiros 80% dos pontos para treino e os últimos 20% para teste.
  Como é série temporal, não faz sentido embaralhar; eu mantenho a ordem do tempo.


In [ ]:
def criar_dataset(serie, janela):
    """
    Recebe um array 1D (serie) e cria um dataset supervisonado usando janela deslizante.

    Para cada posição t, monta:
    - X[t] = serie[t : t + janela]
    - y[t] = serie[t + janela]

    Eu apanhei um pouco na primeira vez que fiz isso porque sempre me confundia
    com os índices, então deixei a função separada para não bagunçar o restante
    do código.
    """
    X = []
    y = []
    for i in range(len(serie) - janela):
        X.append(serie[i : i + janela])
        y.append(serie[i + janela])
    X = np.array(X)
    y = np.array(y)
    return X, y


janela = 24  # conforme comentado acima

X, y = criar_dataset(serie, janela)
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

In [ ]:
# separando em treino (80%) e teste (20%), respeitando a ordem temporal
tamanho_treino = int(len(X) * 0.8)

X_treino = X[:tamanho_treino]
y_treino = y[:tamanho_treino]

X_teste = X[tamanho_treino:]
y_teste = y[tamanho_treino:]

print("Tamanho treino:", X_treino.shape[0])
print("Tamanho teste :", X_teste.shape[0])

# reshape para usar na LSTM: (amostras, passos_de_tempo, features)
# aqui, "features" = 1 porque é uma série univariada
X_treino_lstm = X_treino.reshape((X_treino.shape[0], X_treino.shape[1], 1))
X_teste_lstm = X_teste.reshape((X_teste.shape[0], X_teste.shape[1], 1))

print("X_treino_lstm shape:", X_treino_lstm.shape)
print("X_teste_lstm shape :", X_teste_lstm.shape)

## 4. Baseline ingênuo (Naïve)

Conforme pedido no enunciado, antes de usar rede neural eu preciso ter uma
**técnica ingênua (Naïve)** para servir de baseline.

Aqui eu usei a versão mais simples possível:

> **Previsão = último valor observado** na janela.

Ou seja, para cada amostra de teste, eu pego o último valor de entrada e digo
que a saída vai ser igual a esse valor. Não tem nenhum “aprendizado”, é só uma
regra fixa bem simples.


In [ ]:
# no baseline Naïve, a previsão é simplesmente o último valor da janela
y_pred_naive = X_teste[:, -1]

mse_naive = mean_squared_error(y_teste, y_pred_naive)
mae_naive = mean_absolute_error(y_teste, y_pred_naive)

print(f"Baseline Naïve - MSE: {mse_naive:.4f}")
print(f"Baseline Naïve - MAE: {mae_naive:.4f}")

# só para visualizar rapidamente em um pedaço da série
plt.figure(figsize=(10, 4))
n_plot = 100  # quantidade de pontos para visualizar
plt.plot(y_teste[:n_plot], label="real")
plt.plot(y_pred_naive[:n_plot], label="Naïve", alpha=0.7)
plt.title("Comparação rápida no conjunto de teste (real vs Naïve)")
plt.xlabel("índice no conjunto de teste")
plt.ylabel("valor")
plt.grid(True)
plt.legend()
plt.show()

## 5. Rede neural para forecasting (LSTM simples)

Agora eu uso uma **rede neural recorrente (LSTM)** para tentar melhorar a previsão
em relação ao baseline Naïve.

A ideia é aproveitar as técnicas de forecasting vistas em aula com redes neurais,
mas sem exagerar na complexidade do modelo.

Arquitetura escolhida (bem simples):

- camada `LSTM` com 32 unidades;
- camada `Dense` final com 1 neurônio (previsão escalar);
- função de perda: `mse` (erro quadrático médio);
- métrica: `mae` (erro absoluto médio);
- otimizador: `adam` (funciona bem na maioria dos casos).

Eu não quis tunar demais para não ficar parecendo aqueles exemplos muito “arrumados”.
A ideia é só mostrar que a LSTM consegue aprender algo da estrutura da série.


In [ ]:
modelo_lstm = keras.Sequential([
    layers.Input(shape=(janela, 1)),
    layers.LSTM(32, activation="tanh"),
    layers.Dense(1)
])

modelo_lstm.compile(optimizer="adam", loss="mse", metrics=["mae"])

modelo_lstm.summary()

In [ ]:
# aqui eu faço o treinamento da LSTM
# começo com um número razoável de épocas, mas sem exagerar

historico = modelo_lstm.fit(
    X_treino_lstm,
    y_treino,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## 6. Curvas de treinamento (loss e MAE)

Agora eu ploto as curvas de **loss (MSE)** e **MAE** no treino e na validação.

Isso ajuda a ver se o modelo está:

- diminuindo o erro ao longo das épocas;
- começando a overfittar (quando o erro de validação piora enquanto o de treino melhora);
- ou se já “saturou” (não adianta aumentar muito mais as épocas).

É basicamente a mesma ideia que usei no Trabalho 01, só que aqui com MSE/MAE em série temporal.


In [ ]:
def plotar_curvas_treinamento(historico):
    hist = historico.history
    epocas = range(1, len(hist["loss"]) + 1)

    plt.figure(figsize=(12, 4))

    # Loss (MSE)
    plt.subplot(1, 2, 1)
    plt.plot(epocas, hist["loss"], label="treino")
    plt.plot(epocas, hist["val_loss"], label="validação")
    plt.xlabel("Época")
    plt.ylabel("Loss (MSE)")
    plt.title("Loss - treino vs validação")
    plt.grid(True)
    plt.legend()

    # MAE
    plt.subplot(1, 2, 2)
    plt.plot(epocas, hist["mae"], label="treino")
    plt.plot(epocas, hist["val_mae"], label="validação")
    plt.xlabel("Época")
    plt.ylabel("MAE")
    plt.title("MAE - treino vs validação")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()


plotar_curvas_treinamento(historico)

## 7. Avaliação no conjunto de teste e comparação com o baseline

Agora avalio a LSTM no conjunto de teste e comparo os erros (MSE e MAE) com o
baseline Naïve.

A expectativa é que a LSTM consiga reduzir o erro em relação à regra ingênua
de “prever o último valor observado”.


In [ ]:
# previsões da LSTM no conjunto de teste
y_pred_lstm = modelo_lstm.predict(X_teste_lstm).flatten()

mse_lstm = mean_squared_error(y_teste, y_pred_lstm)
mae_lstm = mean_absolute_error(y_teste, y_pred_lstm)

print(f"Baseline Naïve - MSE: {mse_naive:.4f}, MAE: {mae_naive:.4f}")
print(f"LSTM           - MSE: {mse_lstm:.4f}, MAE: {mae_lstm:.4f}")

In [ ]:
# visualizando um trecho da série de teste para comparar:
plt.figure(figsize=(10, 4))
n_plot = 100
plt.plot(y_teste[:n_plot], label="real")
plt.plot(y_pred_naive[:n_plot], label="Naïve", alpha=0.7)
plt.plot(y_pred_lstm[:n_plot], label="LSTM", alpha=0.7)
plt.xlabel("índice no conjunto de teste")
plt.ylabel("valor")
plt.title("Comparação no conjunto de teste (real vs Naïve vs LSTM)")
plt.grid(True)
plt.legend()
plt.show()

## 8. Conclusão

Neste Trabalho 03, eu:

1. **Gerei uma série temporal sintética** com:
   - leve tendência;
   - componente sazonal aproximadamente com período 24;
   - ruído aleatório moderado.

   Isso me deu uma base simples, mas razoável, para testar forecasting sem
   depender de um dataset externo específico.

2. Transformei a série em um **dataset supervisonado** usando uma
   **janela deslizante** de tamanho 24 (`janela = 24`), justificando esse valor
   pelo fato de a sazonalidade também ter sido construída com período 24.

3. Separei o dataset em treino (80%) e teste (20%), mantendo a ordem temporal
   para não “misturar futuro com passado”.

4. Implementei um **baseline ingênuo (Naïve)** onde a previsão é apenas o
   **último valor observado** na janela. Isso serviu como referência mínima
   de desempenho.

5. Modelei uma rede neural **LSTM simples** (32 unidades + camada densa) para
   tentar capturar melhor a estrutura da série (tendência + sazonalidade).

6. Comparei os erros (MSE e MAE) do baseline Naïve com os da LSTM no conjunto
   de teste e, em geral, a LSTM conseguiu erro menor, mostrando que ela aprendeu
   algo além da regra ingênua.

Os parâmetros principais (como o tamanho da janela e o tamanho da série) foram
escolhidos pensando em:

- respeitar a sazonalidade simulada (24 pontos ≈ 1 ciclo);
- ter dados suficientes para treino e teste (500 pontos totais);
- manter o modelo e o tempo de treinamento em um nível compatível com o
  contexto da disciplina (sem precisar de muitos recursos).

Ainda daria para testar variações de janela, número de épocas, tamanho da LSTM,
etc., mas para o objetivo do trabalho (baseline + técnica de forecasting com rede
neural) considero que os requisitos foram atendidos.
